In [101]:
import pandas as pd
import os
import datetime
import time
import random
import copy

In [102]:
pd.set_option("display.max_columns", None)

In [109]:
data = {}
param = {}
input_directory = '../temp'
param_directory = '../param'
for _filename in os.listdir(input_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(input_directory+'/'+_filename,  header = None)
    filetype = _filename.split('-')[1]
    column_dict = { 
            'sked': ['from', 'to', 'al','fln', 'actype', 'depday','dep','arrday','arr'],
            'comp': ['from', 'to', 'al', 'fln','actype', 'depday','dep','arrday','arr'],
            'demand': ['orig', 'dest', 'unit','unit_vol', 'rev'],
            'demand_curve': ['orig', 'dest', 'ttt_1','ttt_2', 'ttt_3','ttt_4'],          
            'route_cost': ['from','to', 'actype', 'value'],
            'airport_cost': ['ap', 'actype', 'value'],
            'config': ['actype', 'unit_cap', 'vol_cap']
    }
    df_data.columns = column_dict[filetype]
    data[filetype] = df_data
    
for _filename in os.listdir(param_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(param_directory+'/'+_filename,  header = None)
    column_dict = { 
            'network' : ['stops_allowed'],
            'connections': ['hub', 'minct', 'maxct'],
            'preferences' : ['from', 'to', 'stop_penalty'],
    }
    df_data.columns = column_dict[_filename]
    param[_filename] = df_data

test-airport_cost
test-comp
test-config
test-demand
test-demand_curve
test-route_cost
test-sked
connections
preferences


In [110]:
#inputs
your_sked = data['sked']
comp_sked = data['comp']
param_connect = param['connections']
param_pref = param['preferences']
param_pref['od'] = param_pref["from"]+param_pref["to"]
data_config = data['config']

max_stop = 2

your_sked = your_sked.merge(data_config, on='actype')
comp_sked = comp_sked.merge(data_config, on='actype')


full_sked = pd.concat([your_sked,comp_sked])
full_sked["dep"] = pd.to_datetime(full_sked["dep"]) + full_sked["depday"]*datetime.timedelta(days=1)
full_sked["arr"] = pd.to_datetime(full_sked["arr"]) + full_sked["arrday"]*datetime.timedelta(days=1)
full_sked["id"] = full_sked['al'] + full_sked['fln'].astype('str')+"/"+full_sked['depday'].astype('str')
full_sked["unit_load"] = 0
full_sked["vol_load"] = 0
full_sked["netw_rev"] = 0


param_connect['minct'] = pd.to_timedelta(param_connect['minct'])
param_connect['maxct'] = pd.to_timedelta(param_connect['maxct'])

df_connect = full_sked.add_suffix('_1').reset_index(drop=True)
df_connect['nbstops'] = 0

list_itin = {}
list_itin[0] = df_connect.copy()
list_itin[0]['unit_demand']= 0
list_itin[0]['vol_demand']= 0
list_itin[0]['rev']= 0


#build itineraries
for i in range (1,max_stop+1):
    cnx_p = param_connect.copy()
    itins = list_itin[i-1].merge(full_sked.add_suffix('_'+str(i+1)), how= "cross")
    itins['nbstops'] = i
    
    itins['cnx_time_'+str(i)] = itins['dep_'+str(i+1)] - itins['arr_'+str(i)]
    
    cnx_p.columns = ['to_'+str(i), 'minct_'+str(i), 'maxct_'+str(i)]
    
    #filter connections
    itins.drop(itins[itins['to_'+str(i)] != itins['from_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(i)] == itins['to_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(1)] == itins['to_'+str(i+1)]].index, inplace=True)
    
    itins = itins.merge(cnx_p, on = ['to_'+str(i)])
    
    itins.drop(itins[itins['cnx_time_'+str(i)] < itins['minct_'+str(i)]].index, inplace=True)
    itins.drop(itins[itins['cnx_time_'+str(i)] > itins['maxct_'+str(i)]].index, inplace=True)
   
    list_itin[i] = itins


#add attributes
for i in range(0,max_stop+1):
    list_itin[i]["travel_time"] = list_itin[i]["arr_"+str(i+1)]-list_itin[i]["dep_1"]
    list_itin[i]["od"] = list_itin[i]["from_1"]+list_itin[i]["to_"+str(i+1)]
    list_itin[i]["index"] = list_itin[i].index
    list_itin[i]['itin_id'] = list_itin[i]["nbstops"].astype(str)+"-"+list_itin[i]["index"].astype(str)
    list_itin[i] = list_itin[i].merge(param_pref, on="od")

    #calculate score
    list_itin[i]["score"]= list_itin[i]["travel_time"].dt.seconds+list_itin[i]["nbstops"]*list_itin[i]["stop_penalty"]*3600

    


In [111]:
def build_options(list_itin):
    od_itin = {}
    list_itin_summary = pd.DataFrame(columns=["itin_id","index", "od","travel_time", "nbstops","score"])

    for i in range(0,max_stop+1):
        list_itin_summary= pd.concat([list_itin_summary,list_itin[i][["itin_id","index", "od","travel_time", "nbstops","score"]]]) 
    #organize data by OD
    for od in param_pref.od.unique():    
        od_itin[od] = list_itin_summary[list_itin_summary['od']==od].copy()
    return list_itin_summary, od_itin

list_itin_summary, od_itin = build_options(list_itin)

In [112]:
#variables
timep = 4

#define user arrivals
demand_rand = {}
demand_curve = data['demand_curve']
demand = data['demand']
demand['od'] = demand['orig']+demand['dest']
demand_by_ttt = pd.merge(demand, demand_curve, on=["orig","dest"])

for i in range(1,1+timep):
    demand_by_ttt['d_ttt_'+str(i)]=(demand_by_ttt['ttt_'+str(i)]*demand_by_ttt['unit']).astype('int')

#shuffle pax

for i in range(1,1+timep):
    randm = []
    for indx, row in demand_by_ttt.iterrows():
        for k in range(1,row['d_ttt_'+str(i)]):
            randm.append(row['orig']+row['dest'])
    random.shuffle(randm)
    demand_rand[i] = randm
    

In [113]:
spill = dict.fromkeys(demand['od'].unique(),0)

avail_list_itin = {}
for key in list_itin:   
    avail_list_itin[key] = list_itin[key].copy()

avail_od_itin = {}
for key in od_itin:
    avail_od_itin[key] = od_itin[key].copy()

avail_list_itin_summary = list_itin_summary.copy()

###########################################################################################
#########################  TESTING WITH ONLY 1 TIMEPERIOD BUT NEED TO LOOP ################
###########################################################################################
list_pax = demand_rand[1]


#choose an itin randomly among available ones
for od_select in list_pax:
    if len(avail_od_itin[od_select])>0 :
        choice = random.choices(list(avail_od_itin[od_select]["itin_id"]), weights = list(avail_od_itin[od_select]["score"]), k=1)
        stop_index = choice[0].split('-')
        choice_stops = int(stop_index[0])
        choice_index = int(stop_index[1])  
        itin_selected = avail_list_itin[choice_stops]['index']==choice_index
        
        #add pax and revenue to itinerary 
        itin_selected_orig_list = list_itin[choice_stops]['index']==choice_index
        list_itin[choice_stops].loc[itin_selected_orig_list,'unit_demand'] += 1
        list_itin[choice_stops].loc[itin_selected_orig_list,'vol_demand'] += demand.loc[demand['od']==od_select,'unit_vol'].values[0]
        list_itin[choice_stops].loc[itin_selected_orig_list,'rev'] += demand.loc[demand['od']==od_select,'rev'].values[0]
        
        
        #add pax to flights
        for i in range(0, choice_stops+1):
            if len(avail_list_itin[choice_stops].loc[itin_selected,'id_'+str(i+1)])== 0:
                break
            flights_chosen = avail_list_itin[choice_stops].loc[itin_selected,'id_'+str(i+1)].values[0]
            full_sked.loc[full_sked['id']==flights_chosen,'unit_load' ] += 1
            full_sked.loc[full_sked['id']==flights_chosen,'vol_load' ] += demand.loc[demand['od']==od_select,'unit_vol'].values[0]
            full_sked.loc[full_sked['id']==flights_chosen,'netw_rev' ] += demand.loc[demand['od']==od_select,'rev'].values[0]
        
        #remove closed flights
        for i in range(0, choice_stops+1):
            if (full_sked.loc[full_sked['id']==flights_chosen,'unit_load'].values[0] == full_sked.loc[full_sked['id']==flights_chosen,'unit_cap'].values[0] or
                full_sked.loc[full_sked['id']==flights_chosen,'vol_load'].values[0] >= full_sked.loc[full_sked['id']==flights_chosen,'vol_cap'].values[0]  ) :
                for j in range(0, max_stop+1):
                    for k in range(0,j+1):
                        avail_list_itin[j].drop(avail_list_itin[j][avail_list_itin[j]['id_'+str(k+1)] == flights_chosen].index, inplace=True)
                avail_list_itin_summary, avail_od_itin = build_options(avail_list_itin)
                
    else:
        #log spill
        spill[od_select] += 1 
        


In [114]:
avail_list_itin[choice_stops]

,from_1,to_1,al_1,fln_1,actype_1,depday_1,dep_1,arrday_1,arr_1,unit_cap_1,vol_cap_1,id_1,unit_load_1,vol_load_1,netw_rev_1,nbstops,unit_demand,vol_demand,rev,travel_time,od,index,itin_id,from,to,stop_penalty,score


In [115]:
full_sked

,from,to,al,fln,actype,depday,dep,arrday,arr,unit_cap,vol_cap,id,unit_load,vol_load,netw_rev
0,DUS,TXL,FF,1,737,1,2022-02-04 10:52:00,1,2022-02-04 12:24:00,50,570,FF1/1,48,576,480
1,DUS,CDG,FF,2,737,1,2022-02-04 14:14:00,1,2022-02-04 17:00:00,50,570,FF2/1,31,592,570
2,DUS,FRA,FF,3,737,1,2022-02-04 10:49:00,1,2022-02-04 15:37:00,50,570,FF3/1,2,624,46
3,DUS,MAD,FF,5,737,3,2022-02-06 09:24:00,3,2022-02-06 10:25:00,50,570,FF5/3,7,669,278
4,DUS,BCN,FF,6,737,3,2022-02-06 08:58:00,3,2022-02-06 10:23:00,50,570,FF6/3,48,576,20736
5,TXL,DUS,FF,8,737,1,2022-02-04 06:06:00,1,2022-02-04 08:32:00,50,570,FF8/1,0,0,0
6,CDG,DUS,FF,9,737,1,2022-02-04 05:24:00,1,2022-02-04 06:06:00,50,570,FF9/1,0,0,0
7,FRA,DUS,FF,10,737,1,2022-02-04 06:37:00,1,2022-02-04 07:19:00,50,570,FF10/1,0,0,0
8,MAD,DUS,FF,12,737,3,2022-02-06 09:43:00,3,2022-02-06 10:01:00,50,570,FF12/3,0,0,0
9,BCN,DUS,FF,13,737,3,2022-02-06 04:30:00,3,2022-02-06 05:15:00,50,570,FF13/3,0,0,0


In [100]:
list_itin[1][['od','unit_demand', 'vol_demand', 'rev']]

,od,unit_demand,vol_demand,rev
0,DUSTXL,26,312,260
1,DUSTXL,17,204,170
2,LHRMAD,12,504,384
3,LHRMAD,4,168,128
4,LHRMAD,7,294,224
5,TXLMAD,21,252,1575
6,BCNMAD,2,24,172
7,BCNMAD,0,0,0
8,BCNMAD,5,60,430
9,BCNMAD,5,60,430


In [43]:
avail_list_itin[choice_stops]

,from_1,to_1,al_1,fln_1,actype_1,depday_1,dep_1,arrday_1,arr_1,cap_1,weight_1,id_1,load_1,nbstops,demand,from_2,to_2,al_2,fln_2,actype_2,depday_2,dep_2,arrday_2,arr_2,cap_2,weight_2,id_2,load_2,cnx_time_1,minct_1,maxct_1,travel_time,od,index,id,from,to,stop_penalty,score
2,LHR,CDG,AF,17,737,1,2022-02-03 11:10:00,1,2022-02-03 15:31:00,13,57000,AF17/1,0,1,0,CDG,MAD,AF,24,737,1,2022-02-03 16:53:00,1,2022-02-03 17:34:00,13,57000,AF24/1,0,0 days 01:22:00,0 days 00:15:00,0 days 08:00:00,0 days 06:24:00,LHRMAD,25,1-25,LHR,MAD,1,26640


In [44]:
itin_selected

2    False
5     True
Name: index, dtype: bool

In [36]:
itin_selected = avail_list_itin[choice_stops]['index']==choice_index

In [37]:
itin_selected

2    False
Name: index, dtype: bool